In [1]:
import random
import numpy as np
import pickle
import json
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from flask import Flask, render_template, request
from keras.models import load_model


In [2]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/varunprakash/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/varunprakash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/varunprakash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
stop=set(stopwords.words('english'))

In [4]:
ignore_words = ["?", "!", ",",'.']
def clean_up_sentence(sentence):
    sentence_words = word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(w.lower()) for w in sentence_words if w not in ignore_words]
    return sentence_words

In [5]:

def convert_input(description):
    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(description)
    x_description = tokenizer.texts_to_sequences(description)
    return x_description

In [6]:
def nlp_preprocessing(desc):
    corpus = []
    lem = WordNetLemmatizer()

    words = [w for w in nltk.word_tokenize(desc) if (w not in stop)]
    words = [lem.lemmatize(w) for w in words if len(w)>2]
    words = [''.join(c for c in s if c not in string.punctuation) for s in words if s]
    words = [word.lower() for word in words]
    words = [word for word in words if word.isalpha()]
    corpus.append(words) 

    text = []

    for i in range(len(corpus)):
        text.append(' '.join(w for w in corpus[i]))

    return text

In [11]:
def predict_accident_level(desc):
    maxlen = 56
    preprocess = nlp_preprocessing(desc)
    model_input = convert_input(preprocess)
    x = pad_sequences(model_input, padding='post', maxlen=maxlen)
    model = load_model(r"/Users/varunprakash/Downloads/finalized_keras_model_1.h5")
    resp = model.predict(x)
    return np.argmax(resp)             

In [12]:
def predict_potential_accident_level(desc):
    maxlen = 86
    preprocess = nlp_preprocessing(desc)
    model_input = convert_input(preprocess)
    x = pad_sequences(model_input, padding='post', maxlen=maxlen)
    model = load_model('/Users/varunprakash/Downloads/finalized_keras_model_with_target_combine_acc_level.h5')
    resp = model.predict(x)
    return np.argmax(resp)

In [61]:
app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")


@app.route("/bot", methods=["POST"])
def bot_rules():
    
    f = open(r"/Users/varunprakash/Downloads/intents_new.json")
    data = json.load(f)
    
    msg = request.form["msg"]
    
    is_name = False
    name = ''
    
    if msg.startswith('My name is'):
        name = msg[11:]
        is_name = True

    elif msg.startswith('This is'):
        name = msg[8:]
        is_name = True
    
    greetings = data['greetings']
    greetings_response = data['greetings_response']
    
    goodbye = data['goodbye']
    goodbye_response = data['goodbye_response']

    thanks = data['thanks']
    thanks_response = data['thanks_response']
    
    accident = data['accident']
    accident_response = data['accident_response']
    
    default_response = data['default_response']
    
    text = clean_up_sentence(msg)
    print(text)

    if is_name:
        return "Hi {name}, how can i help you".format(name=name)
    
    if any(x in text for x in greetings):
        if is_name:
            return greetings_response_with_name
        else:
            return greetings_response
        
    elif any(x in text for x in goodbye):
        return goodbye_response
    
    elif any(x in text for x in thanks):
        return thanks_response
    
    elif any(x in text for x in accident):
        return accident_response
            
    elif any(x in msg for x in goodbye):
        return goodbye_response
    
    elif any(x in msg for x in thanks):
        return thanks_response
    
    elif any(x in msg for x in accident):
        return accident_response    

    elif len(msg) < 35:
        return default_response
            
    else:
        accident_level = predict_accident_level(msg)
        potential_accident_level = predict_potential_accident_level(msg)
        print('accident_level is', accident_level)
        print('potential_accident_level is', accident_level)
        return data[str(accident_level)] + ' and the Potential accident level is '+ str(potential_accident_level)

if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jan/2022 12:04:49] "POST /bot HTTP/1.1" 200 -


['hi']


127.0.0.1 - - [09/Jan/2022 12:05:45] "POST /bot HTTP/1.1" 200 -


['i', 'want', 'to', 'report', 'an', 'accident']
['at', 'approximately', '6:20', 'p.m.', 'on', '08/06/16', 'the', 'operator', 'eustaquio', 'fall', 'down', 'from', 'the', 'metal', 'platform', 'that', 'give', 'access', 'to', 'tank', 'd-1021', 'of', 'the', 'strong', 'acid', 'leaching', 'stage', 'and', 'suffers', 'luxo-fractures', 'in', 'the', 'wrist', 'left', 'when', 'leaning', 'on', 'the', 'floor', 'with', 'his', 'hand', 'the', 'operator', 'wa', 'directed', 'to', 'the', 'first', 'tank', 'of', 'the', 'strong', 'acid', 'leaching', 'stage', '(', 'tk', 'd-2040', ')', 'to', 'verify', 'the', 'entry', 'of', 'spent', 'to', 'the', 'taque']


...

['hello']


127.0.0.1 - - [09/Jan/2022 12:08:51] "POST /bot HTTP/1.1" 200 -


['i', 'want', 'to', 'know', 'the', 'acci', 'dmaeafdsf']


127.0.0.1 - - [09/Jan/2022 12:13:47] "POST /bot HTTP/1.1" 200 -


['sdfsdf']
